## Libraries

In [ ]:
import qiime2
from tempfile import mkdtemp
from qiime2.plugins import demux, deblur, quality_filter, \
                           metadata, feature_table, alignment, \
                           phylogeny, diversity, emperor, feature_classifier, \
                           taxa, composition
import pandas as pd
import os
import numpy as np
import random
import logging
from datetime import datetime
from Bio import Entrez
from pprint import pprint
from sklearn.utils import shuffle
import xml.etree.ElementTree as ET
import os
from tqdm.notebook import tqdm
from Bio import SeqIO

#### Studies
* exercise_frequency
* flossing_frequency
* vitamin_d_supplement_frequency
* weight_change
* fruit_frequency

## Functions

### Pipeline cleaning

In [2]:
# Puliscel a working directory (result_extraction), elimina tutti i file e le cartelle fatta eccezione per il logfile
# il quale viene solo svuotato
def clean_workspace():
    
    # ci spostiamo nella working directory
    starting_path = os.getcwd()
    path = os.path.join(os.getcwd(), f'result_extraction')
    os.chdir(path)
    
    # prendiamo tutti i file contenuti nella working directory e eliminiamo a meno che non sia il log file
    files = os.listdir()
    for file in files:
        if file != 'sra_querying.log':
            delete_command = f'rm -r {file}'
            os.system(delete_command)
            
    # puliamo il log file
    delete_old_log = 'cat > sra_querying.log'
    os.system(delete_old_log)  
    
    # torniamo nella starting directory ./Human microbiome
    os.chdir(starting_path)

### NCBI utilities

In [3]:
# Funzioni per semplificare l'interrogazioen di NCBI
Entrez.email = "giacomo.villa.mi@gmail.com"

def good_print(text):
    print(pprint(text))

def esearch(db, query, num_max = 20):
    handle = Entrez.esearch(db = db, term = query, retmax = num_max)
    record = Entrez.read(handle, validate = True)
    return record

def esummary(db, id_val):
    handle = Entrez.esummary(db = db, id = id_val)
    record = Entrez.read(handle, validate = True)
    return record

### Final Sample generation

In [4]:
# Prende in input il dataset completo, data la query, 
def write_age_mean(total, typology, experiment):
    
    man = total.query('sex == "male"')
    woman = total.query('sex == "female"')
    
    mean_age_total = round(np.mean(total['age_years']), 4)
    mean_age_man = round(np.mean(man['age_years']), 4)
    mean_age_woman = round(np.mean(woman['age_years']), 4)
    not_valid_sex = total.query("sex != 'female' and sex != 'male'")
    
    if typology != 'sample':
        logging.info(f'--------------------{typology.upper()}--------------------')
    logging.info(f'Total number of {typology} people: {len(total)}')
    logging.info(f'Total number of {typology} man: {len(man)}')
    logging.info(f'Total number of {typology} woman: {len(woman)}')
    logging.info(f'Total number of non valid sex {len(not_valid_sex)}')
    
    logging.info(f'Mean Age for total: {mean_age_total}')
    logging.info(f'Mean Age for man: {mean_age_man}')
    logging.info(f'Mean Age for woman: {mean_age_woman}')
    logging.info(f'\n')
    
    print(f'--------------------{typology.upper()}--------------------')
    print(f'Total number of {typology} people: {len(total)}')
    print(f'Total number of {typology} man: {len(man)}')
    print(f'Total number of {typology} woman: {len(woman)}')
    print(f'Total number of non valid sex {len(not_valid_sex)}')
    
    print(f'Mean Age for total: {mean_age_total}')
    print(f'Mean Age for man: {mean_age_man}')
    print(f'Mean Age for woman: {mean_age_woman}')
    
    starting_path = os.getcwd()
    path = os.path.join(os.getcwd(), f'result_extraction/')
    os.chdir(path)
    directories = os.listdir()
    if f'{experiment}' not in directories:
        os.mkdir(f'{experiment}')
        
    os.chdir(f'{experiment}')
    if typology != 'sample':
        total.to_csv(f"./dataset_query_result_{typology}.csv", index=False, encoding='utf-8')
    
    os.chdir(starting_path)

# Prende in input il dataset da cui campionare e gli index degli element già presi, finché non campiona qualcosa di 
# nuovo non termina
def generate_single_sample(already_taken, from_df):
    element = random.randint(0, len(from_df) - 1)
    while (element in already_taken):
        element = random.randint(0, len(from_df) - 1)
    return element

# Dato il dataset di partenza contenente solo elementi effettivamente presenti su NCBI
# genera un nuovo dataset composto da n_samples elementi
def get_final_sample(started_dataset, n_samples):
    taken = set()
    final_sample = pd.DataFrame(columns=started_dataset.columns)
    for i in tqdm(range(n_samples), desc='Sampling data'):
        new_sample = generate_single_sample(taken, started_dataset)
        taken.add(new_sample)
        final_sample = final_sample.append(started_dataset.iloc[new_sample], ignore_index=True)
    return final_sample

def write_not_valid_ids(not_valid_names):
    files = os.listdir(f'./result_extraction')
    
    if 'not_valid_sample_names.csv' in files:
        all_not_valid_names = pd.read_csv("./result_extraction/not_valid_sample_names.csv", header=0, dtype=str)
        elements = [all_not_valid_names, not_valid_names]
        final_not_valid = pd.concat(elements, ignore_index=False, sort=False)
        final_not_valid.to_csv(f"./result_extraction/not_valid_sample_names.csv", index=False)
    else:
        not_valid_names.to_csv(f"./result_extraction/not_valid_sample_names.csv", index=False)
    
# Dato il dataset di partenza con tutti gli elementi che rispettano la query posta al gut, tiene in considraz
def sampling_data(start_dataset, typology, n_samples, experiment):
    write_age_mean(start_dataset, typology, experiment)
    valid_id = list()
    not_valid_id = list()
    for try_id in tqdm(start_dataset['sample_name'], desc='NCBI ids validation'):
        handleSce = esearch('biosample', try_id)
        if len(handleSce['IdList']) != 0:
            valid_id.append(try_id)
        else:
            not_valid_id.append(str(try_id))
            print(try_id)
            
    print(f'Total rows: {len(start_dataset)}')
    print(f'Valid rows: {len(valid_id)}')
    
    not_valid_names = pd.DataFrame(data={"not_valid_sample_name": not_valid_id, "typology": [typology]*len(not_valid_id)})
    print(len(not_valid_names))
    write_not_valid_ids(not_valid_names)
    
    logging.info(f'Total rows: {len(start_dataset)}')
    logging.info(f'Valid rows: {len(valid_id)}')
            
    valid_start_dataset = pd.DataFrame(columns=start_dataset.columns)
    index = 0
    for _, row in start_dataset.iterrows():
        if row['sample_name'] in valid_id:
            valid_start_dataset.loc[index] = row
            index += 1
            
    valid_start_dataset = shuffle(valid_start_dataset)
    final_sample = get_final_sample(valid_start_dataset, n_samples)
    write_sample_info(final_sample, typology, experiment)

# Funzione richiamata sul sample finale, traduce in csv il campione finale creando un csv di due colonne: sample_name,
# typology (e.g. healthy/not_healthy), richiama write_age_mean per scrivere sul log file l'età media dei soggetti
def write_sample_info(sample, typology, experiment):
    man = sample.query("sex == 'male'")
    woman = sample.query("sex == 'female'")
    write_age_mean(sample, 'sample', experiment)
    
    sample = sample[['sample_name']]
    sample['typology'] = [typology]*len(sample)
    
    files = os.listdir(f'./result_extraction/{experiment}')
    
    if f'final_sample_{experiment}.csv' in files:
        final_sample = pd.read_csv(f"./result_extraction/{experiment}/final_sample_{experiment}.csv", header=0, dtype=str)
        final_sample = final_sample[['sample_name', 'typology']]
        elements = [final_sample, sample]
        final_sample = pd.concat(elements, ignore_index=False, sort=False)
        final_sample.to_csv(f"./result_extraction/{experiment}/final_sample_{experiment}.csv", index=False)
    else:
        sample.to_csv(f"./result_extraction/{experiment}/final_sample_{experiment}.csv", index=False)
        
    close_dashes = '-'*len(typology.upper())
    logging.info(f'--------------------{close_dashes}--------------------')

### SRA operation

In [5]:
# Funzione controller, prende in input il nome dell'esperimento (eg. healthy vs not_healthy) e le tipologie di campione
# (e.g. healthy e not_healty), chiama la funzione che interroga SRA di NCBI, in seguito richiama la funzione per 
# concatenare i file fasta e infine la funzione per prendere le sequenze che ricorrono più spesso. Infine salva su un
# file csv i fasta delle sequenze più popolose aggiungendo il campo che si rifà all'id di Biosample.
# Il file csv delle frequenze più popole è l'input per blast
def sra_querying(experiment, types):
    
    # Legge il csv contente i 30 campioni di una tipologia e i 30 campioni dell'altra tipologia dato l'esperimento
    # (e.g. esperimento: healthy vs not_healthy estrare il csv che contiene i 30 sample_name degli healthy e i 
    # 30 sample_name dei not_healthy)
    final_sample = pd.read_csv(f"./result_extraction/{experiment}/final_sample_{experiment}.csv", header=0, dtype=str)
    
    # crea una lista che conterrà, dati i record, il corrispettivo id di Biosample
    bio_sample_id = list()
    
    # per ogni riga del csv dei 30 campioni di una tipologia e i 30 dell'altra tipologia dato l'esperimento
    for index, row in final_sample.iterrows():
        print(f'File number: {index+1}')
        
        # Gestione del problema sulla lettura di un sample_name con la concatenazione della stringa '001'
        # in generale estrae l'input per la funzione che farà la query su SRA
        record_id = str(row[0])[0:15]
        record_typology = row[1]
        
        # data la singola interrogazione, aggiunge alla lista degli id di bio_sample l'id.
        bio_sample_id.append(get_sequences(record_id, record_typology, experiment))
        
    # una volta scaricate tutti i file fasta data l'esperimento, per ogni tipologia (e.g healthy/not_healthy) 
    # crea un unico file con tutte le sequenze e poi prende, da questo file, solo quelle più popolose
    for typology in types:
        concatenate_fast_file(typology, 'fasta', experiment)
        get_top_sequences(typology, experiment)
        
    # crea una nuova colonna dove, per ogni sample_name, vi sarà l'id di biosample associato e salva il nuovo csv
    final_sample['bio_sample_id'] = bio_sample_id
    final_sample.to_csv(f"./result_extraction/{experiment}/final_sample_{experiment}.csv", index=False)
    
# Funzione che interroga SRA, dato il sample_name. Richiede anche la tipologia del campione (e.g. healthy o not_healthy)
# e il nome dell'esperimento (e.g. healthy_vs_not_healthy), per andare a salvare correttamente nelle cartelle facenti
# riferimento all'esperimento
def get_sequences(sample_name, typology, experiment):
    
    # in funzione della tipologia del campione (e.g. healthy o not_healthy) e dell'esperimento (e.g. healthy_vs_not_healthy)
    # definisce il path corretto dove andare a salvare il risultato
    path = f'"./result_extraction/{experiment}/SRA_{typology}" '
    command1 = f'fastq-dump --fasta --readids --outdir {path}'
    command2 = f'fastq-dump --readids --outdir {path}'
        
    # Query su SRA e print utili
    print(f'Sample id: {sample_name}')
    handleSce = esearch('biosample', sample_name)
    biosampleId = handleSce['IdList'][0]
    print(f'Biosample ID {biosampleId}')
    print(f'Typology: {typology}')
    handleSra = Entrez.efetch(db='biosample', id=biosampleId, retmode='xml')
    root = ET.fromstring(handleSra.read())
    identifier = root.findall('.//BioSample//Ids//Id')
    for i in identifier:
        if i.attrib['db'] == 'SRA':
            sraId = i.text
    print(f'SRA ID: {sraId}')
    handleSra = Entrez.esearch(db='sra', term=sraId)
    resultsSra = Entrez.read(handleSra)['IdList']
    print(resultsSra)
    for s in resultsSra:
        handlesngSraId = Entrez.efetch(db='sra', id=s, retmode='xml')
        root = ET.fromstring(handlesngSraId.read())
        identifier = root.find('.//EXPERIMENT_PACKAGE//RUN_SET//RUN')
        runId = identifier.attrib['accession']
        os.system(command1 + runId)
        os.system(command2 + runId) 
    print()
    return biosampleId
        

# Dato il risultato delle query su SRA, concatena i file fasta facenti riferimento a una certa tipologia di record
# (e.g. healthy o not_healthy) dato un certo esperimento (e.g. healthy_vs_not_healthy)
def concatenate_fast_file(typology, file_format, experiment):
    
    # Prende tutti i file data la tipologia del record (e.g. healthy o not_healthy) contenuti nella cartella dove,
    # dato l'esperimento (e.g. healthy_vs_not_healthy), la query su SRA ha riposto i risultati
    files = os.listdir(f'./result_extraction/{experiment}/SRA_{typology}')
    
    # concatenazione file fasta
    compact_files = list()
    for file in files:
        if file_format in file:
            f = open(f'./result_extraction/{experiment}/SRA_{typology}/{file}', "r")
            compact_files.append(f.read())
            f.close()
    f = open(f'./result_extraction/{experiment}/SRA_{typology}/final_{file_format}_{typology}.{file_format}', 'w')
    for file in compact_files:
        f.write(file)
    f.close()
    
    # Eliminazione dei 
    starting_path = os.getcwd()
    path = os.path.join(os.getcwd(), f'result_extraction/{experiment}/SRA_{typology}')
    os.chdir(path)
    command = 'rm *[0-9].fasta'
    os.system(command) 
    os.chdir(starting_path)
        
def get_top_sequences(typology, experiment):
    records = list(SeqIO.parse(f"./result_extraction/{experiment}/SRA_{typology}/final_fasta_{typology}.fasta", format="fasta"))
    print(f'Number of sequences for {typology}: {len(records)}')
    logging.info(f'Number of sequences for {typology}: {len(records)}')
    
    sequences = dict()
    for record in tqdm(records, desc='Compacting fasta'):
        if record.seq in sequences:
            sequences[record.seq][0] += 1
        else:
            sequences[record.seq] = [1, f'>{record.description}']
    
    print(f'Number of grouped sequences: {len(sequences)}')
    logging.info(f'Number of grouped sequences: {len(sequences)}')
    
    sequences_ord = {k: v for k, v in sorted(sequences.items(), key=lambda item: item[1], reverse=True)}
    
    cont = 0
    f = open(f'./result_extraction/{experiment}/SRA_{typology}/top_sequences_{typology}.fasta', 'w')
    
    for element in sequences_ord:
        if sequences_ord[element][0] >= 100:
            f.write(f'{sequences_ord[element][1]} number of reps {sequences_ord[element][0]}')
            f.write('\n')
            f.write(str(element))
            f.write('\n')
            cont += 1
    f.close()
    
    print(f'Number of taken sequences: {cont}')
    logging.info(f'Number of taken sequences: {cont}')
    print()
    logging.info('\n')

## Main

In [ ]:
# Carica il dataset gut
df = pd.read_csv("./data/american_gut.txt", delimiter="\t", dtype=str)

# Sostituisce con NAN valori non validi
df.replace(' ', np.nan, inplace=True)
df.replace('Not provided', np.nan, inplace=True)
df.replace('Unspecified', np.nan, inplace=True)

# Elimina dalla working directory tutti i risultati dello scorso esperimento, ripulisce il logfile
clean_workspace()

In [ ]:
# Inizializza il log file
logging.basicConfig(filename='./result_extraction/sra_querying.log', level=logging.INFO, format='%(message)s')
today = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
logging.info(f'RUN TIME: {today}')

## All columns

In [ ]:
for column in df.columns:
    print(column, end = ', ')

## Healthy vs not healthy study
### Healthy extraction

In [ ]:
# Estraiamo dal dataset i dati di interesse
healthy = df.query("smoking_frequency == 'Never' and alcohol_frequency == 'Never'")

healthy = healthy.query("cancer == 'I do not have this condition'")

healthy['bmi'] = healthy['bmi'].apply(lambda x: float(x))
healthy = healthy.query("bmi >= 18.5 and bmi <= 24.99")

healthy['age_years'] = healthy['age_years'].apply(lambda x: float(x))
healthy = healthy.query("age_years >= 20 and age_years <= 50")

healthy = healthy.query("body_site == 'UBERON:feces'")

In [ ]:
sampling_data(healthy, 'healthy', 30, 'healthy_vs_not_healthy')

### Not healthy extraction

In [ ]:
not_healthy = df.query("smoking_frequency == 'Occasionally (1-2 times/week)' or smoking_frequency == 'Daily' or smoking_frequency == 'Regularly (3-5 times/week)'")
not_healthy = not_healthy.query("alcohol_frequency == 'Occasionally (1-2 times/week)' or alcohol_frequency == 'Daily' or alcohol_frequency == 'Regularly (3-5 times/week)'")

not_healthy = not_healthy.query("cancer == 'I do not have this condition'")

not_healthy['bmi'] = not_healthy['bmi'].apply(lambda x: float(x))
not_healthy = not_healthy.query("bmi < 18.5 or bmi > 24.99")

not_healthy['age_years'] = not_healthy['age_years'].apply(lambda x: float(x))
not_healthy = not_healthy.query("age_years >= 20 and age_years <= 50")

not_healthy = not_healthy.query("body_site == 'UBERON:feces'")

In [ ]:
sampling_data(not_healthy, 'not_healthy', 30, 'healthy_vs_not_healthy')

## Mental illness vs food disorders
### Mental illness

In [ ]:
mental_illness = df.query("country_residence == 'United States'")

mental_illness = mental_illness.query("body_site == 'UBERON:feces'")

mental_illness = mental_illness.query("mental_illness == 'true' or mental_illness == 'Yes'")

mental_illness['age_years'] = mental_illness['age_years'].apply(lambda x: float(x))

In [ ]:
sampling_data(mental_illness, 'mental_illness', 30, 'mental_ill_vs_food_dis')

### Food disorders

In [ ]:
food_disorders = df.query("country_residence == 'United States'")

food_disorders = food_disorders.query("body_site == 'UBERON:feces'")

food_disorders = food_disorders.query("mental_illness == 'false' or mental_illness == 'No'")

food_disorders['bmi'] =  food_disorders['bmi'].apply(lambda x : float(x))
food_disorders = food_disorders.query("bmi < 18.5 or bmi > 24.99")

food_disorders = food_disorders.query("(fruit_frequency == 'Never' or fruit_frequency == 'Rarely (less than once/week)')")

food_disorders = food_disorders.query("exercise_frequency=='Rarely (a few times/month)' or exercise_frequency=='Never'")

food_disorders['age_years'] = food_disorders['age_years'].apply(lambda x: float(x))

In [ ]:
sampling_data(food_disorders, 'food_disorders', 30, 'mental_ill_vs_food_dis')

## NCBI Quering

In [6]:
sra_querying('mental_ill_vs_food_dis', ['mental_illness', 'food_disorders'])

# 7497069, 14618671

File number: 1
Sample id: 10317.000067678
Biosample ID 7497069
Typology: mental_illness
SRA ID: ERS1865634
['4376103']

File number: 2
Sample id: 10317.000089968
Biosample ID 8945722
Typology: mental_illness
SRA ID: ERS2409408
['5424517']

File number: 3
Sample id: 10317.000102683
Biosample ID 14619503
Typology: mental_illness
SRA ID: ERS4424358
['10592811']

File number: 4
Sample id: 10317.000084671
Biosample ID 8576788
Typology: mental_illness
SRA ID: ERS2212798
['5144054']

File number: 5
Sample id: 10317.000074845
Biosample ID 9541547
Typology: mental_illness
SRA ID: ERS2483510
['5840876']

File number: 6
Sample id: 10317.000072390
Biosample ID 8728606
Typology: mental_illness
SRA ID: ERS2303239
['5245093']

File number: 7
Sample id: 10317.000022557
Biosample ID 9541095
Typology: mental_illness
SRA ID: ERS2483066
['5840372']

File number: 8
Sample id: 10317.000049819
Biosample ID 5265530
Typology: mental_illness
SRA ID: ERS1208079
['2650193']

File number: 9
Sample id: 10317.000036


Number of grouped sequences: 211619
Number of taken sequences: 550

Number of sequences for food_disorders: 1549729



Number of grouped sequences: 867911
Number of taken sequences: 564



In [7]:
sra_querying('healthy_vs_not_healthy', ['healthy', 'not_healthy'])

File number: 1
Sample id: 10317.000093022
Biosample ID 9653255
Typology: healthy
SRA ID: ERS2606638
['5959856']

File number: 2
Sample id: 10317.000058340
Biosample ID 6367687
Typology: healthy
SRA ID: ERS1564754
['3738232']

File number: 3
Sample id: 10317.000107254
Biosample ID 14618791
Typology: healthy
SRA ID: ERS4423949
['10592389']

File number: 4
Sample id: 10317.000032796
Biosample ID 5158596
Typology: healthy
SRA ID: ERS1164403
['4933109', '2559054']

File number: 5
Sample id: 10317.000046476
Biosample ID 7353916
Typology: healthy
SRA ID: ERS1821824
['4279996']

File number: 6
Sample id: 10317.000079781
Biosample ID 8569142
Typology: healthy
SRA ID: ERS2203316
['5125489']

File number: 7
Sample id: 10317.000040396
Biosample ID 6364886
Typology: healthy
SRA ID: ERS1561319
['3735156', '3723191']

File number: 8
Sample id: 10317.000069615
Biosample ID 8614819
Typology: healthy
SRA ID: ERS2215183
['5162969']

File number: 9
Sample id: 10317.000050243
Biosample ID 6365125
Typology:


Number of grouped sequences: 1184492
Number of taken sequences: 547

Number of sequences for not_healthy: 858682



Number of grouped sequences: 260130
Number of taken sequences: 464

